In [ ]:
import sys
import os
sys.path.append('../utils/analysis/')

import pandas as pd
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns
from cycler import cycler

from filtering import File_Filter, Subject_Filter

In [ ]:
# Load chartevents using File_Filter
file_filter = File_Filter("chartevents")

# Load the lookup table to get subject IDs
lookup_df = pd.read_csv('../data/icu_unique_subject_ids.csv')

In [ ]:
# Get a random subject ID
random_subject = random.choice(lookup_df['subject_id'].tolist())
print(f"Selected random subject: {random_subject}")

# Load the chartevents data for this subject
chartevents_df = file_filter.search_subject(random_subject)
print(f"Loaded {len(chartevents_df)} rows for subject {random_subject}")
print(chartevents_df.head())

In [ ]:
# From chartevents_df, filter "valueuom" column, specifically rows with value "bpm"
bpm_df = chartevents_df[chartevents_df['valueuom'] == 'bpm']
bpm_df.head()

In [ ]:
# Prepare data
bpm_df = bpm_df.copy()
bpm_df['charttime'] = pd.to_datetime(bpm_df['charttime'])
bpm_df = bpm_df.sort_values('charttime')

# Calculate day numbers starting from Day 1
bpm_df['day'] = (bpm_df['charttime'] - bpm_df['charttime'].min()).dt.days + 1

plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linestyle'] = '-'
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.prop_cycle'] = cycler(color=['#2563EB', '#E11D48', '#D97706', '#10B981', '#8B5CF6', '#F472B6'])

# Calculate statistics
mean_hr = bpm_df['valuenum'].mean()
min_hr = bpm_df['valuenum'].min()
max_hr = bpm_df['valuenum'].max()
std_hr = bpm_df['valuenum'].std()
sample_count = len(bpm_df)

# Create the plot
plt.figure(figsize=(12, 6))
line, = plt.plot(bpm_df['charttime'], bpm_df['valuenum'], linewidth=2.5)

# Set titles and labels
plt.suptitle('Heart Rate Over Time', fontsize=18, fontweight='bold', ha='center')
plt.title(f'Subject ID: {random_subject}', fontsize=14, ha='center')
plt.xlabel('Time', fontsize=12, fontweight='bold')
plt.ylabel('Heart Rate (bpm)', fontsize=12, fontweight='bold')

# Set tick label sizes and custom x-ticks for days
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
# Get unique days and their earliest charttime
unique_days = sorted(bpm_df['day'].unique())
tick_positions = []
tick_labels = []
for day in unique_days:
    day_start = bpm_df[bpm_df['day'] == day]['charttime'].min()
    tick_positions.append(day_start)
    tick_labels.append(f'Day {day}')
plt.xticks(tick_positions, tick_labels, rotation=45)

# Customize spines
ax = plt.gca()
ax.spines['bottom'].set_linewidth(0.8)
ax.spines['left'].set_linewidth(0.8)
ax.spines['bottom'].set_color('#374151')
ax.spines['left'].set_color('#374151')

# Customize grid
ax.grid(color='#E5E7EB', linestyle='-', linewidth=0.5, alpha=0.5)

# Add legend with statistics
plt.legend([line], [f'Mean: {mean_hr:.1f} bpm\nMin: {min_hr:.0f} bpm\nMax: {max_hr:.0f} bpm\nStd: {std_hr:.1f} bpm\nSample Count: {sample_count}'], loc='upper right')

plt.show()

In [ ]:
# 12323950
# 15621838